In [89]:
import math
import numpy as np
from shapely.geometry import MultiPoint
from itertools import product, chain, pairwise

In [90]:
breakpoints = [([-116.276208, 686.644227], 0),
([-43.256272, 704.133424], 0),
([-97.187671, 928.036095], 0),
([-30.823008, 943.939461], 0),
([-22.791816, 910.584813], 0),
([41.779946, 926.056381], 0),
([129.221734, 546.278255], 0),
([-37.647863, 363.520713], 0),
([-101.383804, 618.480078], 0)]



Cxy = list(range(58, 109))
Ch = [9, 12]

In [91]:
class BaseSection:
    def __init__(self, angle, leng, widt, road):
        self.angle = angle
        self.an_normalized = self.angle_normalized()
        self.length, self.width, self.road = leng, widt, road

    def angle_normalized(self):
        if self.angle > math.pi / 4:
            if self.angle < math.pi / 2:
                ang = (math.pi / 2) - self.angle
            elif 3 * math.pi / 4 > self.angle > math.pi / 2:
                ang = self.angle - (math.pi / 2)
            else:
                ang = math.pi - self.angle
        else:
            ang = self.angle
        return ang

In [92]:
class PolyClusters:
    def __init__(self, split_points, cells, road):
        self.split_points = split_points
        self.cells = cells
        self.road = road
        self.bound_rec = np.asarray(MultiPoint([i[0] for i in self.split_points]).minimum_rotated_rectangle.exterior.coords)
       self.projection = self.points_projected()
        #self.grids = self.grid_options()
        #self.distances, self.coordinates = self.distances()
        #self.coordinates_dist = self.coordinates_dist()



    #считаем точки спроецированные на баундинг ректангл
    def points_projected(self):
        points = []
        for i, v in enumerate(self.bound_rec[0:4]):
            if i != len(self.bound_rec)-1:
                temp =[]
                length = np.sum((v-self.bound_rec[i+1])**2)
                for sp in self.split_points:
                    if sp[1] == 1:
                        t = max(0, min(1, np.sum((np.asarray(sp[0]) - v) * (self.bound_rec[i+1] - v)) / length))
                        projection = np.asarray(v + t * (self.bound_rec[i+1] - v))
                        temp.append(projection)
                    else:
                        pass
                points.append(np.asarray(sorted(temp, key=lambda k: [k[1], k[0]])))
        return np.asarray(points)



    #считпем расстояния между точками и убиваем мелкие недоразумения вне зависимости от веса. Сейчас 58, возможно надо сделать ключем
    #возвращаем длины и тюпли точек из которых можно сразу считать вектора'''

    def distances(self):
        arr = np.delete(self.projection, -1, axis=1)
        shift_arr = np.delete(np.roll(self.projection, -1, axis=1), -1, axis=1)
        dist = np.linalg.norm(arr-shift_arr, axis = 2 )
        values = []
        distance = []

        for p, i in enumerate(dist):
            values_to_del = []
            d =[]
            sum = 0
            for v in range(len(i)):

                if i[v] < 98 and v!=len(i)-1:
                    sum+=i[v]
                    values_to_del.append(v+1)

                elif i[v] < 98 and v==len(i)-1:
                    sum+=i[v]
                    d[-1]+=sum
                    values_to_del.append(v)

                else:
                    d.append(sum + i[v])
                    sum = 0
            values.append(d)
            clear_list = np.delete(self.projection[p], values_to_del, axis=0)
            distance.append(np.stack((np.delete(clear_list, -1, axis=0), np.delete(np.roll(clear_list, -1, axis=0), -1, axis=0)), axis = 1))

        return values, np.asarray(distance)



    #комбинации ячеек
    def find_numbers(self, ans, temp, dist, index):
        if dist == 0:
            ans.append(list(temp))
        for i in range(index, len(self.cells)):
            if (dist - self.road -self.cells[i]) >= 0:
                temp.append([self.road//2,self.cells[i],self.road//2])
                self.find_numbers(ans, temp, (dist - self.road -self.cells[i]), i)
                temp.remove([self.road//2,self.cells[i],self.road//2])

    def grid_options(self, dist):
        ans = []
        temp = []
        self.find_numbers(ans, temp, dist, 0)
        combinations = []
        for i in ans:
            combinations.append([item for sublist in i for item in sublist])
        return combinations

    #функция просчета ячеек будет внутри расчета координат, чтобы не терять структуру
    def coordinates_dist(self):
        fin =[]
        for i,v in enumerate(self.distances[0:2]):
            coordinates =[]
            for ii, vv in enumerate(v):
                kk =[]
                options = self.grid_options(round(vv))
                vec_one = self.coordinates[i][ii][1] - self.coordinates[i][ii][0]
                vec_two = self.coordinates[i+2][ii][1] - self.coordinates[i+2][ii][0]
                for o in options:
                    dist_l = 0
                    opt = []
                    for oo in o:
                        dist_l+=oo
                        p_one = self.coordinates[i][ii][0] + vec_one / np.linalg.norm(vec_one)*dist_l
                        p_two = self.coordinates[i+2][ii][0] + vec_two / np.linalg.norm(vec_two)*dist_l
                        opt.append([p_one.tolist(),p_two.tolist()])
                    kk.append(opt)
                coordinates.append(kk)

            fin.append(list(product(*coordinates)))
        j = list(product(fin[0], fin[1]))

        return j








In [93]:
p_c = PolyClusters(breakpoints, Cxy, 40)
v = p_c.bound_rec
#list(product(*v[0]))
v.tolist()

[[-168.99420679403624, 912.1263157830638],
 [-42.427458888215924, 362.420237796794],
 [160.781406482799, 409.20794174552753],
 [34.21465857697868, 958.9140197317973],
 [-168.99420679403624, 912.1263157830638]]